# VOCS v1 data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [2]:
?VOCS

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables=None)

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': None}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs


True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives['c']  == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": null}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'definitions': {'ConstraintEnum': {'title': 'ConstraintEnum',
   'description': 'An enumeration.',
   'enum': ['LESS_THAN', 'GREATER_THAN'],
   'type': 'string'},
  'ObjectiveEnum': {'title': 'ObjectiveEnum',
   'description': 'An enumeration.',


# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data['c']  = data['a'] + data['b']
data['d']  = data['a'] - data['b']
data['e']  = data['a']*2 + data['b']*2
data['f']  = data['a']*2 - data['b']*2
data.index = np.arange(len(data)) + 5 # custom index
data


,a,b,g,c,d,e,f
5,529.157867,0.040652,1234,529.198519,529.117215,1058.397038,1058.234430
6,299.889602,0.019515,1234,299.909118,299.870087,599.818236,599.740174
7,140.338930,-0.786521,1234,139.552409,141.125451,279.104818,282.250903
8,25.428540,0.589057,1234,26.017596,24.839483,52.035192,49.678966
9,679.590150,-0.881955,1234,678.708195,680.472105,1357.416389,1360.944210
10,675.259366,0.996994,1234,676.256360,674.262371,1352.512720,1348.524743
11,85.297707,0.823399,1234,86.121106,84.474307,172.242212,168.948614
12,256.235692,-0.905126,1234,255.330566,257.140817,510.661131,514.281635
13,940.613591,0.826587,1234,941.440178,939.787004,1882.880357,1879.574008
14,298.302483,-0.084514,1234,298.217969,298.386998,596.435938,596.773996


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-529.198519,529.117215
6,-299.909118,299.870087
7,-139.552409,141.125451
8,-26.017596,24.839483
9,-678.708195,680.472105
10,-676.256360,674.262371
11,-86.121106,84.474307
12,-255.330566,257.140817
13,-941.440178,939.787004
14,-298.217969,298.386998


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-529.198519,529.117215
6,-299.909118,299.870087
7,-139.552409,141.125451
8,-26.017596,24.839483
9,-678.708195,680.472105
10,-676.256360,674.262371
11,-86.121106,84.474307
12,-255.330566,257.140817
13,-941.440178,939.787004
14,-298.217969,298.386998


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-529.1985189 ,  529.11721524],
       [-299.90911794,  299.870087  ],
       [-139.55240885,  141.12545148],
       [ -26.01759621,   24.83948299],
       [-678.70819475,  680.47210506],
       [-676.25636004,  674.26237129],
       [ -86.121106  ,   84.4743072 ],
       [-255.33056574,  257.1408173 ],
       [-941.44017836,  939.78700405],
       [-298.21796914,  298.38699778]])

In [17]:
?form_constraint_data

In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1056.397038,-1058.234430
6,597.818236,-599.740174
7,277.104818,-282.250903
8,50.035192,-49.678966
9,1355.416389,-1360.944210
10,1350.512720,-1348.524743
11,170.242212,-168.948614
12,508.661131,-514.281635
13,1880.880357,-1879.574008
14,594.435938,-596.773996


In [19]:
?form_feasibility_data

In [20]:
vocs.feasibility_data(data)

,feasibility_e,feasibility_f,feasibility
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False
